In [52]:
import pandas
import requests
import csv

In [120]:
def filter_authors():
    url = "https://lobid.org/gnd/search"
    filter_query = """
        (
            (professionOrOccupation.id:"https://d-nb.info/gnd/4053311-6") 
                OR
            (professionOrOccupation.id:"https://d-nb.info/gnd/4053309-8")
        )
        # &(type:Person)
        # &(type:DifferentiatedPerson)
        &(dateOfBirth:[1750 TO 1870])
    """
    params = {
        'q': '*',
        'filter': filter_query,
        'from' : '24501',
        'size' : '500',
        'format': 'json'
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status() 
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        return None
    
res = filter_authors()


Error making request: 500 Server Error: Internal Server Error for url: https://lobid.org/gnd/search?q=%2A&filter=%0A++++++++%28%0A++++++++++++%28professionOrOccupation.id%3A%22https%3A%2F%2Fd-nb.info%2Fgnd%2F4053311-6%22%29+%0A++++++++++++++++OR%0A++++++++++++%28professionOrOccupation.id%3A%22https%3A%2F%2Fd-nb.info%2Fgnd%2F4053309-8%22%29%0A++++++++%29%0A++++++++%23+%26%28type%3APerson%29%0A++++++++%23+%26%28type%3ADifferentiatedPerson%29%0A++++++++%26%28dateOfBirth%3A%5B1750+TO+1870%5D%29%0A++++&from=24501&size=500&format=json


In [118]:
import csv

with open('temp3.csv', 'w', encoding='utf-8', newline='') as csv_file:
    # Define CSV column headers
    fieldnames = ['GND', 'Nachname', 'Vorname', 'DateOfBirth', 'DateOfDeath', 'Gender', 'Professions', 'VariantNames']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    for member in res['member']:
        # Extract fields safely with proper handling for the structure
        gnd = "".join(member.get('gndIdentifier', ''))
        name = member.get('preferredNameEntityForThePerson', {}).get('surname', '')  # For 'Nachname'
        vorname = member.get('preferredNameEntityForThePerson', {}).get('forename', '')  # For 'Vorname'
        date_of_birth = ", ".join(member.get('dateOfBirth', []))
        date_of_death = ", ".join(member.get('dateOfDeath', []))
        gender = ", ".join([g['label'] for g in member.get('gender', [])])
        professions = ", ".join([p['label'] for p in member.get('professionOrOccupation', [])])
        variant_names = ", ".join([
            " ".join(v.get('forename', [])) + " " + 
            " ".join(v.get('prefix', [])) + " " + 
            " ".join(v.get('surname', []))
            for v in member.get('variantNameEntityForThePerson', [])
        ]).strip()

        # Write a row for the current member
        writer.writerow({
            'GND': gnd,
            'Nachname': name,
            'Vorname': vorname,
            'DateOfBirth': date_of_birth,
            'DateOfDeath': date_of_death,
            'Gender': gender,
            'Professions': professions,
            'VariantNames': variant_names
        })


In [121]:
import pandas as pd
import os
folder_path = r'temp'
all_files = os.listdir(folder_path)
csv_files = [f for f in all_files if f.endswith('.csv')]
df_list = []
for csv in csv_files:
    file_path = os.path.join(folder_path, csv)
    try:
        df = pd.read_csv(file_path)
        df_list.append(df)
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(file_path, sep='\t', encoding='utf-16')
            df_list.append(df)
        except Exception as e:
            print(f"Could not read file {csv} because of error: {e}")
    except Exception as e:
        print(f"Could not read file {csv} because of error: {e}")
big_df = pd.concat(df_list, ignore_index=True)
big_df.to_csv(os.path.join(folder_path, 'combined_file.csv'), index=False)